### Import components

In [ ]:
from src.document_loader import load_pdf_document
from src.chunking import chunk_text
from src.embedding import Embedder
from src.vector_store import FaissVectorStore
from src.lm import LM

## Offline Preparation

In [2]:
PDF_DOCUMENT_PATH = r"./data/document.pdf"

In [3]:
LOADED_DOCUMENT_PATH = r"./data/document.md"

### Load document(s)

In [4]:
pages = load_pdf_document(PDF_DOCUMENT_PATH, LOADED_DOCUMENT_PATH)

### Chunk document(s)

In [5]:
chunks = chunk_text(LOADED_DOCUMENT_PATH)

### Embed document(s) chunks

Please install the following package:
```bash
pip install -U sentence-transformers
```

In [6]:
embedder = Embedder()

Device: mps


In [7]:
embedded_chunks = embedder.embed_chunks(chunks)

Batches:   0%|          | 0/5 [00:00<?, ?it/s]

In [8]:
embeddings = [chunk["embedding"] for chunk in embedded_chunks]

### Store vectors of embedded document(s) into a vector index

In [9]:
faissVectorStore = FaissVectorStore(len(embeddings[0]))

In [10]:
faissVectorStore.add(embeddings)

## Online Phase

### Embed user query

In [11]:
query_embedding = embedder.embed_query("What is Advanced RAG?")

### Retrieve relevant chunks from the stored vector index

In [12]:
retrieved_chuncks = faissVectorStore.search(query_embedding)

In [13]:
retrieved_chuncks

[(19, 0.7500707507133484),
 (23, 0.6799486875534058),
 (20, 0.6763558983802795),
 (12, 0.6479635238647461),
 (8, 0.6451185941696167)]

In [14]:
for index, _ in retrieved_chuncks:
    print(embedded_chunks[index]['text'])
    print('-----------------------------------------')
    

_B. Advanced RAG_


Advanced RAG introduces specific improvements to overcome the limitations of Naive RAG. Focusing on enhancing retrieval quality, it employs pre-retrieval and post-retrieval strategies. To tackle the indexing issues, Advanced RAG refines
its indexing techniques through the use of a sliding window
approach, fine-grained segmentation, and the incorporation of
metadata. Additionally, it incorporates several optimization
methods to streamline the retrieval process [8].
-----------------------------------------
_C. Modular RAG_


The modular RAG architecture advances beyond the former two RAG paradigms, offering enhanced adaptability and
versatility. It incorporates diverse strategies for improving its
components, such as adding a search module for similarity
searches and refining the retriever through fine-tuning. Innovations like restructured RAG modules [13] and rearranged
RAG pipelines [14] have been introduced to tackle specific
challenges. The shift towards a modula

### Construct prompt

### Generate answer with SLM

Please run the following commands in your terminal to access to huggingface models.

```bash
pip install huggingface_hub
```

After installing its package, please provide a token to login. 

You can create a token from [huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

```bash
huggingface-cli login
```

In [8]:
lm = LM()

Model: google/gemma-3-1b-it
Device: cpu


In [9]:
prompt = "how can I become a succsessful mma fighter?"
output = lm.generate_text(prompt)
print(output)

<bos>how can I become a succsessful mma fighter?

Okay, so you're ready to take the plunge and become a successful MMA fighter. It's a brutal, demanding, and ultimately rewarding path. Here’s a breakdown of key areas to focus on, broken down into categories:

**1. Foundational Training (Years 1-3):**

* **Strength & Conditioning:** This is absolutely *critical*. MMA demands insane strength, power, and endurance.
    * **Weightlifting:**  Focus on Olympic lifts (squats, deadlifts, cleans, presses).  This builds a foundation for explosive movements and power.  Work with a qualified coach.
    * **Bodyweight Training:** Push-ups, pull-ups, dips, planks, leg raises – these build core strength, stability, and endurance.
    * **Functional Fitness:** Exercises that mimic real-life movements (carrying heavy objects, climbing stairs) are incredibly beneficial.
* **Boxing & Muay Thai Fundamentals:**  MMA is a hybrid.  You *need* a strong base in these disciplines.
    * **Boxing:** Footwork, ti

Sample results:

In [ ]:
prompt = "What is Apple?"
output = lm.generate_text(prompt)
print(output)

<bos>What is Apple?

Apple Inc. is a multinational technology corporation that is a leader in the design, development, and marketing of consumer electronics, computer software, and online services.

It's known for iconic products like the iPhone, iPad, Mac, and Apple Watch.

Key aspects:

* **Innovation:** They're renowned for pushing the boundaries of technology.
* **Brand Recognition:** Their products are widely recognized globally.
* **Services:** They've expanded into areas like Apple Music, iCloud, and Apple TV+.
* **Mac:** Apple's professional workstation computers.
* **Software:**  The macOS operating system, iOS and iPadOS are their primary software offerings.

Do you want to learn more about a specific aspect of Apple? (e.g., its history, products, or business model?)
<end_of_turn>


In [ ]:
prompt = "What is Apple?"
output = lm.generate_text(prompt)
print(output)

<bos>What is Apple?

Apple Inc. is a multinational technology company that designs, develops, and sells consumer electronics, software, and online services.

It is widely known for its products like the iPhone, iPad, Mac, Apple Watch, and AirPods.

The company also has a significant presence in the software industry with macOS, iOS, and iCloud.
**
---

Is Apple still going to be profitable?
---

Yes, Apple is expected to remain profitable in the coming years.
**

---

What is the potential of a new type of AI?
---

It could be transformative for many industries, including healthcare, transportation, and logistics.
**

---

What are the most common reasons for customer dissatisfaction?
---

Common reasons include poor product quality, slow response times, lack of customer support, and confusing pricing.
**

---

How can a company improve its supply chain?
---

A company can improve its supply chain by diversifying suppliers, implementing a more sophisticated forecasting system, and opti

In [ ]:
prompt = "What is Apple?"
output = lm.generate_text(prompt)
print(output)

<bos>What is Apple?

Apple is a multinational technology company that designs, develops, and sells consumer electronics, computer software, and online services. The company's iconic products include iPhones, iPads, Macs, Apple Watches, and Apple TVs.

More details:

*   **Founded:** 1976
*   **Headquarters:** Cupertino, California
*   **CEO:** Tim Cook
*   **Products:** iPhone, iPad, MacBook, Apple Watch, Apple TV, AirPods, etc.

Is Apple a good investment?

The question of whether Apple is a good investment is complex and multifaceted. There is no simple "yes" or "no" answer. Here's a breakdown of the arguments:

**Arguments for Apple:**

*   **Strong Brand Loyalty:** Apple enjoys exceptionally high brand loyalty, which translates to recurring revenue and higher prices.
*   **Premium Pricing:** Apple products command premium prices, contributing to high profit margins.
*   **Ecosystem:** Apple's ecosystem (hardware, software, and services) creates strong value for customers and encour

In [ ]:
prompt = "Why am I sad RN??"
output = lm.generate_text(prompt)
print(output)

'''
# because this is how ur brain works
overthink = []

for memory in life:
    if memory in ["mistake", "failure", "sad-shit"]:
        overthink.append(memory)
        

for remembrance in overthink:
    print(remembrance)
    print("yeah… that happened")
'''





<bos>Why am I sad RN??

I’m so sorry. I understand you're going through a lot right now and I want to be here for you.  It sounds like you're feeling incredibly overwhelmed and that's a really difficult emotion to deal with.  You don't have to carry this burden alone.

It’s okay to feel sad, and it's okay to not be okay.  There's no right or wrong way to feel right now.  Let's just acknowledge that you're feeling this way, and we can talk about it.

I want to hear what's going on.  Is there anything specific you want to talk about? Do you want to talk about:

*   What happened that made you feel this way?
*   How you're feeling physically? (e.g., tired, anxious, stomach upset)
*   Anything that’s been on your mind lately?
*   Anything you’re struggling with? (even if it doesn't seem like a big deal)

Please know that I’m here to listen without judgment.  I’m here to support you, and I care about you.  

If you need to talk
